In [ ]:
import sys
import numpy as np
from tqdm.auto import tqdm
np.set_printoptions(precision=2, linewidth=200, suppress=True)

from sklearn.decomposition import PCA

from itertools import permutations, combinations
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from scipy.spatial.distance import pdist, squareform

num_points = 24
dim = 2
max_error = 1e-4

In [ ]:
# Function to generate random points in 2D space
def generate_random_points(num_points, space_size=(100, 100)):
    return np.random.rand(num_points, 2) * space_size

def initialize_grid(num_points, grid_size=100):
    """
    Initialize a grid pattern with a quadratic number of points.

    Parameters:
    - num_points (int): Total number of points, must be a perfect square.
    - grid_size (float): The size of the grid (length of one side).

    Returns:
    - points (np.ndarray): A numpy array of shape (num_points, 2) representing the grid points.
    """
    # Check if num_points is a perfect square
    grid_side = int(np.sqrt(num_points))
    if grid_side ** 2 != num_points:
        raise ValueError("num_points must be a perfect square")

    # Create the grid points
    x = np.linspace(0, grid_size, grid_side)
    y = np.linspace(0, grid_size, grid_side)
    xv, yv = np.meshgrid(x, y)
    
    # Combine the x and y coordinates into a single array
    points = np.column_stack([xv.ravel(), yv.ravel()])
    
    return points

In [ ]:
def iter_triu_indices(n, k=1):
    a1, a2 = np.triu_indices(n, k=k)
    return np.flip(n - 1 - a2), np.flip(n - 1 - a1)

# Function to calculate pairwise Euclidean distances
def distance_matrix(points):
    return np.sqrt(np.sum((points[:, np.newaxis, :] - points[np.newaxis, :, :]) ** 2, axis=-1))


def calculate_distances(points):
    distances = distance_matrix(points)
    i, j = np.triu_indices(points.shape[0], 1)
    return -np.sort(-distances[i, j])
    
#@jit
def loss_function(points, target_distances):
    """Calculate the mean squared error between sorted guessed distances and sorted target distances."""
    distances = calculate_distances(points)
    return np.mean((distances - target_distances) ** 2)

In [ ]:
# Function to align point cloud to a standard reference frame
def align_to_reference_frame(points, return_fn=False):
    # Calculate the center of the point cloud
    centroid = np.mean(points, axis=0)
    
    # Translate the point cloud to the origin
    translated_points = points - centroid
    
    # Perform PCA to find the principal components
    pca = PCA(n_components=2)
    pca.fit(translated_points)
    
    # Rotate points so that the largest eigenvector aligns with the x-axis
    rotated_points = pca.transform(translated_points)

    def transform_points(points):
        return np.concatenate([pca.transform(points[:, :len(centroid)] - centroid), points[:, len(centroid):]], axis=-1)

    if return_fn:
        return rotated_points, transform_points
    else:
        return rotated_points

In [ ]:
# Classical MDS
def mds(distance_matrix, n, dim=2):
    D_squared = distance_matrix ** 2
    J = np.eye(n) - np.ones((n, n)) / n
    B = -0.5 * J @ D_squared @ J
    eigenvalues, eigenvectors = np.linalg.eigh(B)
    return eigenvectors[:, -dim:] @ np.diag(np.sqrt(eigenvalues[-dim:]))

In [ ]:
# Initialize 10 random points
original_points = generate_random_points(num_points)
#original_points = initialize_grid(num_points)

original_points = align_to_reference_frame(original_points)
print("Original Points:", original_points.shape)
print(original_points)
print()

# Convert their absolute coordinates into relative distances
sorted_distances = calculate_distances(original_points)
print(sorted_distances)

# Trigonometry with checking

In [ ]:
def calculate_third_points_vectorized(d1, d2, max_distance):
    """Calculate the possible positions for the third points using vectorized operations."""
    x1 = max_distance / 2
    x2 = -max_distance / 2
    
    # Calculate the x-coordinate of the third points
    x3 = (d1**2 - d2**2) / (2 * max_distance)
    
    # Calculate the y-coordinate squared
    y3_squared = d1**2 - (x3 + x1)**2
    
    # Mask out invalid solutions (negative y3_squared)
    mask = y3_squared >= 0
    x3 = x3[mask]
    y3 = np.sqrt(y3_squared[mask])
    
    return np.column_stack((x3, y3)), mask

def pairs_of_pairs(valid_edges):
    triangle_distance_indices, candidate_distance_indices = [], []
    
    for k, edge1 in enumerate(valid_edges): #tqdm(enumerate(valid_edges), total=len(valid_edges)):
        for l, edge2 in enumerate(valid_edges):
            if np.any(edge1[np.newaxis, :] == edge2[:, np.newaxis]): continue
            yield k, l

In [ ]:
def find_quadrilateral(sorted_distances):
    max_distance = sorted_distances[0]
    
    # Generate all pairs of distances
    i, j = np.triu_indices(len(sorted_distances) - 1, k=1)
    i += 1
    j += 1
    d1, d2 = sorted_distances[i], sorted_distances[j]
    
    possible_positions, mask = calculate_third_points_vectorized(d1, d2, max_distance)
    #print(possible_positions)
    valid_edges = np.stack([i[mask], j[mask]], axis=1)
    
    reconstructed_points = np.stack([[max_distance / 2, 0], [-max_distance / 2, 0]])
    
    if len(reconstructed_points) == 2:
        for k, l in pairs_of_pairs(valid_edges):
            match_indices = np.setdiff1d(np.arange(len(sorted_distances) - 1) + 1, np.concatenate([valid_edges[k], valid_edges[l]]))
            
            pos1, pos2 = possible_positions[k], possible_positions[l]
            pos2 = np.stack([pos2, np.stack([-pos2[0], pos2[1]], axis=-1), np.stack([pos2[0], -pos2[1]], axis=-1), -pos2])
            
            #print('pos1\n', pos1)
            #print('pos2\n', pos2)
            
            possible_distances = np.sqrt(np.sum((pos1[np.newaxis, :] - pos2) ** 2, axis=-1))
            #print('possible_distances\n', possible_distances)
            
            #print('sorted_distances\n', sorted_distances[match_indices])
            distance_errors = np.abs(possible_distances - sorted_distances[match_indices][:, np.newaxis])
            #print('distance_errors\n', distance_errors)
            
            smallest_index = np.argmin(distance_errors)
            smallest_indices = np.unravel_index(smallest_index, distance_errors.shape)
            error = distance_errors[smallest_indices]
            #print(smallest_error)
            
            if error < max_error: break
        
        #print(smallest_indices)
        #print(valid_edges[k], valid_edges[l], match_indices[smallest_indices[0]])
        
        reconstructed_point1 = pos1
        reconstructed_point2 = pos2[smallest_indices[1]]
        
        reconstructed_points = np.concatenate([reconstructed_points, [reconstructed_point1, reconstructed_point2]])
        
        reconstructed_indices = np.array([0, valid_edges[k][1], valid_edges[l][1 - smallest_indices[1] % 2], valid_edges[k][0], valid_edges[l][smallest_indices[1] % 2], match_indices[smallest_indices[0]]])
    
    return reconstructed_points, reconstructed_indices, error

In [ ]:
reconstructed_points, reconstructed_indices, error = find_quadrilateral(sorted_distances)
error

In [ ]:
reconstructed_points

In [ ]:
sorted_distances[reconstructed_indices]

In [ ]:
distance_matrix(reconstructed_points)

In [ ]:
rotated_reconstructed_points = align_to_reference_frame(reconstructed_points)

plt.figure(figsize=(15, 15))
plt.axis('equal')

plt.scatter(original_points[..., 0], original_points[..., 1], 32.0, 'g')

plt.scatter(rotated_reconstructed_points[..., 0], rotated_reconstructed_points[..., 1], 4.0, 'b')
plt.show()

In [ ]:
new_mask = np.logical_and(
    np.logical_and.reduce(i[:, np.newaxis] != reconstructed_indices[1:][np.newaxis, :], axis=-1),
    np.logical_and.reduce(j[:, np.newaxis] != reconstructed_indices[1:][np.newaxis, :], axis=-1)
)
possible_positions = possible_positions[new_mask[mask]]
valid_edges = valid_edges[new_mask[mask]]

mask = np.logical_and(mask, new_mask)

In [ ]:
a1, a2 = iter_triu_indices(4)
print(a1, a2)

In [ ]:
pos1 = reconstructed_points[2]

for k, pos2 in enumerate(possible_positions):
    pos2 = np.stack([pos2, np.stack([-pos2[0], pos2[1]], axis=-1), np.stack([pos2[0], -pos2[1]], axis=-1), -pos2])
    
    possible_distances = np.sqrt(np.sum((pos1[np.newaxis, :] - pos2) ** 2, axis=-1))
    print('possible_distances\n', possible_distances)
    
    print('sorted_distances\n', sorted_distances)
    distance_errors = np.abs(possible_distances - sorted_distances[:, np.newaxis])
    print('distance_errors\n', distance_errors)
    
    smallest_index = np.argmin(distance_errors)
    smallest_indices = np.unravel_index(smallest_index, distance_errors.shape)
    error = distance_errors[smallest_indices]
    print(error)
    
    if error < max_error: break

In [ ]:
reconstructed_point = pos2[smallest_indices[1]]

In [ ]:
reconstructed_point

In [ ]:
reconstructed_points = np.concatenate([reconstructed_points, [reconstructed_point]])

In [ ]:
reconstructed_points

In [ ]:
quadrilateral_pointss = []
quadrilateral_indicess = []
quadrilateral_errors = []

remaining_sorted_distances = sorted_distances.copy()
original_indices = np.arange(len(remaining_sorted_distances))

while len(remaining_sorted_distances) >= 6:
    print('original_indices\n', original_indices)
    print('remaining_sorted_distances\n', remaining_sorted_distances)
    
    quadrilateral_points, quadrilateral_indices, error = find_quadrilateral(remaining_sorted_distances)
    quadrilateral_pointss.append(quadrilateral_points)
    quadrilateral_indicess.append(quadrilateral_indices)
    quadrilateral_errors.append(error)

    print('quadrilateral_indices\n', quadrilateral_indices)
    remaining_indices = np.setdiff1d(np.arange(len(remaining_sorted_distances)), quadrilateral_indices[1:])
    original_indices = original_indices[remaining_indices]
    remaining_sorted_distances = remaining_sorted_distances[remaining_indices]

In [ ]:
quadrilateral_points = np.concatenate([quadrilateral_pointss[0]] + [points[2:] for points in quadrilateral_pointss[1:]])
quadrilateral_points

In [ ]:
calculate_distances(reconstructed_points)

In [ ]:
sorted_distances

# Try out distance matrices

In [ ]:
def generate_distance_matrices(sorted_distances, n):
    # Create a template matrix with zeros on the diagonal
    template = np.zeros((n, n))
    
    # Get the indices for the upper triangle (excluding diagonal)
    indices = list(zip(*np.triu_indices(n, k=1)))

    shuffled_distances = sorted_distances.copy()
    np.random.shuffle(shuffled_distances)
    
    # Generate all permutations of the distances
    for perm in permutations(shuffled_distances):
        # Create a new matrix for each permutation
        matrix = template.copy()
        
        # Fill the upper triangle with the permuted distances
        for (i, j), dist in zip(indices, perm):
            matrix[i, j] = dist
        
        # Make the matrix symmetric
        matrix = matrix + matrix.T
        
        yield matrix

In [ ]:
def reconstruct_points(sorted_distances, n, max_attempts=1, verbose=False):
    lowest_loss = np.inf
    reconstructed_points = None

    i = 0
    try:
        for distance_matrix in generate_distance_matrices(sorted_distances, n):
            points = mds(distance_matrix, n=n)
            loss = loss_function(points, sorted_distances)
        
            if loss < lowest_loss:
                lowest_loss = loss
                reconstructed_points = points
                if verbose:
                    print(loss)
        
                if lowest_loss < max_error: break
    
            i += 1
            if i == max_attempts: break
    except KeyboardInterrupt:
        if verbose: print('Keyboard Interrupt', file=sys.stderr)
    
    return reconstructed_points, lowest_loss

In [ ]:
#reconstructed_points, loss = reconstruct_points(sorted_distances, num_points, verbose=True, max_attempts=None)

# MDS

In [ ]:
known_combinations = {}
original_indices = np.argsort(np.argsort(distance_matrix(original_points)[np.triu_indices(num_points, k=1)]))
current_indices = original_indices # np.arange(len(sorted_distances))
np.random.shuffle(current_indices)
#current_indices = np.array([20, 15, 25, 18, 27,  0,  4,  9, 12, 24, 10, 17, 11,  3,  6,  7, 14,  5, 13,  1, 23, 16, 22, 19,  8, 26, 21,  2])
sorted_distances

In [ ]:
for i, j in combinations(range(len(current_indices)), 2):
    current_indices = original_indices.copy()
    
    current_indices[i], current_indices[j] = current_indices[j], current_indices[i]
    
    while tuple(current_indices) not in known_combinations:
        current_distances = sorted_distances[current_indices]
        current_dist_matrix = squareform(current_distances)
        
        reconstructed_points = mds(current_dist_matrix, n=num_points)
        reconstructed_dist_matrix = distance_matrix(reconstructed_points)
        reconstructed_distances = reconstructed_dist_matrix[np.triu_indices(num_points, k=1)]
        reconstructed_sorted_distances = np.sort(reconstructed_distances)
        
        loss = loss_function(reconstructed_points, sorted_distances)
        known_combinations[tuple(current_indices)] = loss
        print(loss)

    break
        #current_indices = np.argsort(np.argsort(reconstructed_distances))

    break
    print()

In [ ]:
original_indices = np.argsort(np.argsort(distance_matrix(original_points)[np.triu_indices(num_points, k=1)]))
current_indices = original_indices.copy()
#np.random.shuffle(current_indices)

#current_indices[0], current_indices[3] = current_indices[3], current_indices[0]

current_distances = sorted_distances[current_indices]
current_dist_matrix = squareform(current_distances)
#current_dist_matrix[0][1] += 0.1
#current_dist_matrix[1][0] += 0.1
current_dist_matrix

In [ ]:
n = num_points
D_squared = current_dist_matrix ** 2
J = np.eye(n) - np.ones((n, n)) / n
B = -0.5 * J @ D_squared @ J
eigenvalues, eigenvectors = np.linalg.eigh(B)
eigenvalues

In [ ]:
eigenvectors

In [ ]:
correction = 2 * eigenvalues[0] * eigenvectors[:, 0:1] @ eigenvectors[:, 0:1].T  + 2 * eigenvalues[1] * eigenvectors[:, 1:2] @ eigenvectors[:, 1:2].T
correction

In [ ]:
arbitrary_2 = eigenvalues[2] * eigenvectors[:, 2:3] @ eigenvectors[:, 2:3].T
arbitrary_3 = eigenvalues[3] * eigenvectors[:, 3:4] @ eigenvectors[:, 3:4].T

In [ ]:
X = np.stack([np.diag(arbitrary_2), np.diag(arbitrary_3)], axis=1)
X

In [ ]:
b = np.diag(correction).T
b

In [ ]:
improvements = np.linalg.pinv(X) @ b * 0.0
improvements[0], improvements[1]

In [ ]:
arbitrary_2, arbitrary_3, correction

In [ ]:
optimized_correction = correction - improvements[0] * arbitrary_2 - improvements[1] * arbitrary_3
optimized_correction

In [ ]:
D_corrected = D_squared + optimized_correction
corrected_dist_matrix = np.sign(D_corrected) * np.sqrt(np.abs(D_corrected))
corrected_dist_matrix

In [ ]:
corrected_dist_matrix - current_dist_matrix

In [ ]:
sorted_distances, current_indices

In [ ]:
current_dist_matrix

In [ ]:
reconstructed_points = eigenvectors[:, -dim:] @ np.diag(np.sqrt(eigenvalues[-dim:]))
reconstructed_points

In [ ]:
B_corrected = B - 0.5 * optimized_correction
eigenvalues_corrected, eigenvectors_corrected = np.linalg.eigh(B_corrected)
eigenvalues_corrected

In [ ]:
reconstructed_points_corrected = eigenvectors_corrected[:, -dim:] @ np.diag(np.sqrt(eigenvalues_corrected[-dim:]))
reconstructed_points_corrected

In [ ]:
reconstructed_points = align_to_reference_frame(reconstructed_points)
#reconstructed_points_corrected = align_to_reference_frame(reconstructed_points_corrected)

print("Reconstructed Points:")
print(reconstructed_points)

#for i, step_points in enumerate(intermediate_steps):
#    intermediate_steps[i] = align_to_reference_frame(step_points)
    
plt.figure(figsize=(15, 15))
plt.axis('equal')

plt.scatter(original_points[..., 0], original_points[..., 1], 32.0, 'g')

# Visualize the intermediate steps
#for i, step in enumerate(intermediate_steps):
#    plt.scatter(step[:, 0], step[:, 1], 1.0, 'r', alpha=i / len(intermediate_steps))

plt.scatter(reconstructed_points[..., 0], reconstructed_points[..., 1], 4.0, 'b')
#plt.scatter(reconstructed_points_corrected[..., 0], reconstructed_points_corrected[..., 1], 4.0, 'r')
plt.show()

In [ ]:
current_distances = sorted_distances[current_indices]
current_dist_matrix = squareform(current_distances)

In [ ]:
current_distances = sorted_distances[current_indices]
current_dist_matrix = squareform(current_distances)

reconstructed_points = mds(current_dist_matrix, n=num_points)
reconstructed_dist_matrix = distance_matrix(reconstructed_points)
reconstructed_distances = reconstructed_dist_matrix[np.triu_indices(num_points, k=1)]
reconstructed_sorted_distances = np.sort(reconstructed_distances)

loss = loss_function(reconstructed_points, sorted_distances)
known_combinations[tuple(current_indices)] = loss
loss

In [ ]:
reconstruction_error = current_distances - reconstructed_distances
reconstruction_error

In [ ]:
gradients = sorted_distances - reconstructed_sorted_distances
sort_indices = np.argsort(current_distances)
unsort_indices = np.argsort(sort_indices)
unsorted_gradients = gradients[unsort_indices]
unsorted_gradients

In [ ]:
reconstruction_error - unsorted_gradients

In [ ]:
min_indices = np.argsort(unsorted_gradients)
#max_indices = np.flip(min_indices[len(min_indices)//2:])
#min_indices = min_indices[:len(min_indices)//2]

In [ ]:
min_indices

In [ ]:
current_indices

In [ ]:
current_indices[min_indices], current_indices[max_indices] = current_indices[max_indices], current_indices[min_indices]

In [ ]:
current_indices

In [ ]:
for combination, loss in known_combinations.items():
    print(combination, loss)

In [ ]:
def reconstruct_points(sorted_distances, n, verbose=False):
    distances = sorted_distances.copy()
    try:
        for _ in range(10):...
        distance_matrix = squareform(distances)
        reconstructed_points = mds(distance_matrix, n=n)
        loss = loss_function(points, sorted_distances)
    
        if verbose: print(loss)
        if loss < max_error: break

    except KeyboardInterrupt:
        if verbose: print('Keyboard Interrupt', file=sys.stderr)
    
    return reconstructed_points

# Triangles

In [ ]:
def triangle_inequality(a, b, c):
    return a + b >= c and b + c >= a and c + a >= b

def find_valid_triangles(sorted_distances):
    n = len(sorted_distances)
    for i, j, k in combinations(reversed(range(n)), 3):
        a, b, c = sorted_distances[i], sorted_distances[j], sorted_distances[k]
        if triangle_inequality(a, b , c):
            yield (i, j, k), np.sort([a, b, c])

In [ ]:
sorted_distances

In [ ]:
triplets = []

n = 3
for indices, triangle in find_valid_triangles(sorted_distances):
    print(indices, triangle)
    reconstructed_points, error = reconstruct_points(triangle, n)
    triplets.append(reconstructed_points)
    triplets.append(-reconstructed_points)
    break

In [ ]:
n = 4  # We're looking for 4 points
subset_size = 6

for distances_subset in combinations(sorted_distances, subset_size):
    print(distances_subset)
    if all(triangle_inequality(distances_subset[x], distances_subset[y], distances_subset[z]) for x in range(4) for y in range(x+1, 5) for z in range(y+1, 6)):
        print(True)
    else:
        print(False)
    
    # Reconstruct points using this subset of distances
    distances_subset = np.array(distances_subset)
    reconstructed_points, error = reconstruct_points(distances_subset, n)

    print(error)
    if error < max_error: break

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
def find_next_point(partial_solution, all_distances, error_threshold=1e-6):
    """
    Try to find the next point to add to the partial solution.
    
    :param partial_solution: Tuple of (points, distances) for the current partial solution
    :param all_distances: List of all distances in the original problem
    :param error_threshold: Maximum allowed error for a successful addition
    :return: Tuple of (new_points, new_distances, success)
    """
    points, accounted_distances = partial_solution
    n = len(points)
    
    # Find distances that are not yet accounted for
    unaccounted_distances = set(all_distances) - set(accounted_distances)
    
    # Try adding a new point at different positions
    for new_distances in combinations(unaccounted_distances, n):
        new_point = trilaterate(points, new_distances)
        if new_point is not None:
            new_points = np.vstack((points, new_point))
            new_all_distances = pdist(new_points)
            
            if is_valid_merge(new_all_distances, all_distances, error_threshold):
                new_accounted_distances = set(accounted_distances) | set(new_distances)
                return new_points, list(new_accounted_distances), True
    
    return None, None, False

def trilaterate(points, distances):
    """
    Find a new point given its distances to existing points.
    This is a simplified trilateration and might not work in all cases.
    """
    A = 2 * (points[1:] - points[0])
    b = distances[0]**2 - np.sum(points[0]**2) + np.sum(points[1:]**2, axis=1) - np.array(distances[1:])**2
    try:
        return np.linalg.lstsq(A, b, rcond=None)[0]
    except np.linalg.LinAlgError:
        return None